In [110]:
# my params

Q3a_N=139318878921153532751499562761861928446507405098649008398219509064685868413912856527950494349759160792127809267494353909062242919460654617558230464503523072853970051251389009657389828676782408653856765866847406701730174973965838054507050235650693427895360582038941078988286943128651239704127657219089609607351
Q3a_L=[71428917482626692140169982067862643920420275674669283364745508522809226069141807246524012607121901689983549896702173005394057157599972774611925964105467629064863660890458158323855562269186549518715877956626640975766345665444607083997104539023122776747077177264746821943616443140923900587609056528445625902170, 71428917482626692140169982067862643920420275674669283364745508522809226069141807246524012607121901689983549896702173005394057157599972774611925964105467629064863660890458158323855562269186549518715877956626640975766345665444607083997104539023122776747077177264746821943616443140923900587609056528445625892864, 127928008217276346167336361577656149625995083920413330033640857276488463844161655245001647533047423098521725503859508223718509333968445586126099209962932287246287906410910906759923368271156613625465043826961541638989424258943723093052260395099615387150476380996054707929519308722244956640308340832093776543744]
Q3a_y=19735889032053983154057868876073728216664903734291099482618569146771209013497758865267510952598203047756225193926234102261865551167876941715818029461871888765765060092513537402467007462085036102443934130209613019358047841116721788655675144946688361076515275797917029535600762271496234408860381764181628353362


## Factor N ##

In [173]:
# get last element of the list of points

last_elem_bits = Q3a_L[-1].bits()

# brute force last 15 bits until we find a nontrivial root of 1

bitlen = len(last_elem_bits)
nontrivial_root = 0

for n in range(0, 2**15):
    
    bits = Integer(n).bits()
    
    L = len(bits)
    padded_bits = bits + ([0] * (15 - L))
    
    attempt = padded_bits + last_elem_bits[15:] 
    assert len(attempt) == bitlen
    int_val = ZZ(attempt, base=2)
    square = power_mod(int_val, 2, Q3a_N)
    if square == 1:
        nontrivial_root = int_val
        break
        

    
print("Nontrivial root of 1 is ", nontrivial_root)
print("Root squared mod N is ", power_mod(nontrivial_root, 2, Q3a_N))

# find GCD of N and the nontrivial root minus 1

nontrivial_factor = GCD(Q3a_N, nontrivial_root - 1)
P = nontrivial_factor
Q = ZZ(Q3a_N / P)

print("First nontrivial factor of N is ", P)
print("Is P prime?  ", P.is_prime())
print("Second nontrivial factor of N is ", Q)
print("Is Q prime?  ", Q.is_prime())

print("Do the two nontrivial factors found multiply to N?  ", P*Q == Q3a_N)

# compute

Nontrivial root of 1 is  127928008217276346167336361577656149625995083920413330033640857276488463844161655245001647533047423098521725503859508223718509333968445586126099209962932287246287906410910906759923368271156613625465043826961541638989424258943723093052260395099615387150476380996054707929519308722244956640308340832093776545405
Root squared mod N is  1
First nontrivial factor of N is  13169420695771863846677571050430875414415295065454283167144895221620406447131662042099750516480336983695453551203674736697999446989548142691858914162956767
Is P prime?   True
Second nontrivial factor of N is  10578967909035121752358381405508655860431656165486745446246696102991019817035008002078381129073245564467182462917312046774196639554629163104731621051866153
Is Q prime?   True
Do the two nontrivial factors found multiply to N?   True


## Find quadratic residues with Tonelli's algorithm ##

In [172]:
print()
print("Finding QRs mod P and Q...")

Zn = Zmod(Q3a_N)
Zp = Zmod(P)
Zq = Zmod(Q)

QR_modP = power_mod(Q3a_y, (P+1)/4, P) # square_root_mod_prime(Q3a_y, p=P)
other = sqrt(Zp(Q3a_y))

assert power_mod(QR_modP, 2, P) == (Q3a_y % P)
assert other == QR_modP

QR_modQ = sqrt(Zq(Q3a_y))

assert power_mod(QR_modP, 2, P) == (Q3a_y % P)
assert power_mod(QR_modQ, 2, Q) == (Q3a_y % Q)

print("QR of y mod P is ", QR_modP)
print("QR of y mod Q is ", QR_modQ)

print()
print("Combining QRs of Y with CRT...")
print()

root00 = crt(ZZ(QR_modP), ZZ(QR_modQ), P, Q)
root01 = crt(ZZ(QR_modP), ZZ(Zq(Q - QR_modQ)), P, Q)
root10 = crt(ZZ(Zp(P - QR_modP)), ZZ(Zq(QR_modQ)), P, Q)
root11 = crt(ZZ(Zp(P - QR_modP)), ZZ(Zq(Q - QR_modQ)), P, Q)

rootsY = [root00, root01, root10, root11]

assert(root00 != root01)
assert(root01 != root10)
assert(root10 != root11)

assert(len(set(rootsY)) == 4)

assert power_mod(root01, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root10, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root10, 2, Q3a_N) == Zn(Q3a_y)
assert power_mod(root11, 2, Q3a_N) == Zn(Q3a_y)

for i in range(4):
    print(f"Root number {i} of Y is:")
    print(rootsY[i])
    print()


Finding QRs mod P and Q...
QR of y mod P is  103346012058626652677862313476730297922987086136184769876785561952040555384044923302685866248843953302112471781446570296510955044943426511201176195506531
QR of y mod Q is  935366332157473761671052076845425084918182432539906338474585184867731388160894437427634236724101097241042826928861921230216521140125618481823410672715526

Combining QRs of Y with CRT...

Root number 0 of Y is:
18706770755650775278330499303158974781573545913523812768447827121508365350505516774120279449502720513851332579179557193133216334299794619216671269639234897708489644380118674468968561061607311287203333989944941524522601643179238638725655909049965802701502058744035773972193557734354399576031406929679691302594

Root number 1 of Y is:
50398837482754204648560358825426954729756633712151033363496131184158604102214666246728632726782789370908196347991824447410489105562274620252046098287282788929021941713084049913093636114118676847836149663321619584748215407364189842632888271